# Investigation: AWS IAM S3Browser Templated S3 Bucket Policy Creation

This notebook assists in investigating this detection.

In [ ]:
from scanner_client import ScannerClient
import pandas as pd
import plotly.express as px

# Initialize Scanner Client
client = ScannerClient(api_url="https://api.kast-apse1.scanner.dev", api_key="YOUR_API_KEY")

In [ ]:
query = """
%ingest.source_type="aws:cloudtrail"
eventSource="iam.amazonaws.com"
eventName="PutUserPolicy"
userAgent: 'S3 Browser'
requestParameters.policyDocument: (
  "<YOUR-BUCKET-NAME>"
  and 's3:GetObject'
  and 'Allow'
)
| stats
  min(@scnr.datetime) as firstTime,
  max(@scnr.datetime) as lastTime,
  count() as eventCount
  by
  userIdentity.arn,
  eventSource,
  eventName,
  awsRegion
"""

results = client.query(query, time_range_s=86400)
df = pd.DataFrame(results)
df.head()